# Data Preprocessing and NLP Analysis

In [1]:
# Importing the Raw Data

import duckdb

con = duckdb.connect("madoc.db")

con.execute("""
CREATE TABLE madoc AS
SELECT 
    *,
    split_part(filename, '_', 1) AS platform,
    split_part(filename, '_', 2) AS community
FROM read_parquet('D:/Varun Kumar/Proejekt/Data/raw/*.parquet', filename=true);
""")


In [2]:
# Converting date column type

con.execute("""
ALTER TABLE madoc ADD COLUMN publish_ts TIMESTAMP;
""")

con.execute("""
UPDATE madoc 
SET publish_ts = to_timestamp(publish_date);
""")

con.execute("""
ALTER TABLE madoc ADD COLUMN year INTEGER;
""")

con.execute("""
UPDATE madoc 
SET year = EXTRACT(year FROM publish_ts);
""")


In [3]:
df = con.execute(
    """SELECT * FROM madoc"""
)

In [4]:
# Dropping unused/unnecessary columns

con.execute("""
ALTER TABLE madoc
DROP COLUMN filename;
""")

con.execute("""
ALTER TABLE madoc
DROP COLUMN platform_1;
""")

con.execute("""
ALTER TABLE madoc
DROP COLUMN community_1;
""")

con.execute("""
ALTER TABLE madoc
DROP COLUMN strict_filter;
""")


In [5]:
con.table("madoc").show()

┌──────────────────────────────────────┬──────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [6]:
con.execute("DESCRIBE madoc").df()


,column_name,column_type,null,key,default,extra
0,post_id,VARCHAR,YES,None,None,None
1,publish_date,BIGINT,YES,None,None,None
2,user_id,VARCHAR,YES,None,None,None
3,parent_id,VARCHAR,YES,None,None,None
4,parent_user_id,VARCHAR,YES,None,None,None
5,content,VARCHAR,YES,None,None,None
6,url,VARCHAR,YES,None,None,None
7,language,VARCHAR,YES,None,None,None
8,interaction_type,VARCHAR,YES,None,None,None
9,platform,VARCHAR,YES,None,None,None


In [7]:
con.execute("ALTER TABLE madoc DROP COLUMN publish_date;")

In [ ]:
# Exporting the cleaned data

con.execute("""
COPY madoc TO 'madoc_all.parquet'
(FORMAT PARQUET, COMPRESSION ZSTD);
""")

In [8]:
con.execute("SELECT * FROM madoc WHERE platform = 'voat' AND content IS NOT NULL").df().shape


(714230, 13)

In [9]:
con.execute("SELECT COUNT(*) FROM madoc WHERE platform = 'reddit' AND content IS NOT NULL").df()


,count_star()
0,19960037


### Why we create this 200k sample

The MADOC dataset contains ~20 million posts, but Reddit heavily dominates the data (≈20M vs ≈700k Voat). Running deep-learning models like DistilBERT on the full dataset is not feasible, and using the raw distribution would bias any model toward Reddit.

To enable fair cross-platform NLP analysis, we create a balanced sample of 200,000 posts:

100,000 randomly sampled from Reddit

100,000 randomly sampled from Voat

This sample is large enough to be statistically meaningful, small enough to process on a GPU, and balanced so that Reddit and Voat contribute equally. The full dataset remains in DuckDB for large-scale trend analysis, while this sample is used for expensive NLP tasks such as BERT sentiment and toxicity scoring.

In [13]:
con.execute("""
COPY (
    -- Wrap the filter in a subquery so sampling happens on the RESULT of the filter
    SELECT * FROM (
        SELECT * FROM madoc 
        WHERE platform = 'reddit' AND content IS NOT NULL
    ) USING SAMPLE reservoir(100000)

    UNION ALL

    SELECT * FROM (
        SELECT * FROM madoc 
        WHERE platform = 'voat' AND content IS NOT NULL
    ) USING SAMPLE reservoir(100000)
) TO 'ml_sample_200k.parquet'
(FORMAT PARQUET, COMPRESSION ZSTD);
""")

In [14]:
import pandas as pd

sample = pd.read_parquet("ml_sample_200k.parquet")
sample.shape

(200000, 13)

In [15]:
sample['platform'].value_counts()

platform
reddit    100000
voat      100000
Name: count, dtype: int64